In [23]:
import pandas as pd

# Yıllar ve ilgili HS sürümleri
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı 
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"
ulkeler_df = pd.read_excel('ulkeler_with_codes.xlsx')

# Yeni ürünlerin bulunduğu Excel dosyasını yükle
new_products_df = pd.read_excel('2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')
new_products_df['Yıl'] = new_products_df['Yıl'].ffill()

# 2012 ve 2017'ye ait yeni ürünleri ayır
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]
new_products_2022_df = new_products_df[new_products_df['Yıl'] == 2022]

# Sonuçları depolamak için boş DataFrame
sonuclar3 = pd.DataFrame(columns=['Year', 'Country'])

# Her ülke ve her yıl için işlem yapma
for index, row in ulkeler_df.iterrows():
    ulke = row['Countries']
    ulke_kodu = row['country_code']

    for year in yillar:
        hs_version = hs_versiyonlari[year]
        dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)
        df = pd.read_csv(dosya_yolu)

        # Yalnızca ilgili ülkeye ait veriyi filtreleme
        ulke_veri = df[df['i'] == ulke_kodu]
        toplam_ticaret_hacmi = ulke_veri['v'].sum()

        # Yeni ürünlerin kodlarını al ve ticaret hacimlerini hesapla
        if year <= 2016:
            yeni_urunler = new_products_2012_df['code']
        else:
            yeni_urunler = new_products_2017_df['code']

        # İlk olarak (yıl, ülke) kombinasyonunu DataFrame'e ekliyoruz
        if not ((sonuclar3['Year'] == year) & (sonuclar3['Country'] == ulke)).any():
            sonuclar3 = pd.concat([sonuclar3, pd.DataFrame({'Year': [year], 'Country': [ulke]})], ignore_index=True)

        # Ticaret hacmi hesaplama
        for urun_kodu in yeni_urunler:
            urun_ticaret_hacmi = ulke_veri[ulke_veri['k'] == urun_kodu]['v'].sum()

            if toplam_ticaret_hacmi > 0:
                ticaret_hacmi_orani = (urun_ticaret_hacmi / toplam_ticaret_hacmi) * 100
            else:
                ticaret_hacmi_orani = 0

            # Ürün kodunu iki basamağa indir
            urun_kodu_degeri = str(urun_kodu // 1000).zfill(2)

            # Ürün koduna göre sütun oluşturup veriyi ekle
            if urun_kodu_degeri not in sonuclar3.columns:
                sonuclar3[urun_kodu_degeri] = None

            sonuclar3.loc[(sonuclar3['Year'] == year) & (sonuclar3['Country'] == ulke), urun_kodu_degeri] = ticaret_hacmi_orani

# Sonuçları yazdırma
print(sonuclar3)

# Sonuçları Excel dosyasına kaydetme
sonuclar3.to_excel('e.xlsx', index=False)

# Veriyi uzun formata dökün ve işlemi devam ettirin
df_cleaned = sonuclar3.fillna(0)

df_melted = pd.melt(df_cleaned, id_vars=['Year', 'Country'], var_name='Product', value_name='Value')
df_panel = df_melted.sort_values(by=['Year', 'Country']).reset_index(drop=True)
df_panel.to_excel("d.xlsx", index=False)

# Load the 'd.xlsx' file
d_df = pd.read_excel('d.xlsx')
combined_atlas_df = pd.read_excel('combined_atlas_data.xlsx')

# Step 1: Divide 'Unnamed: 2' column by 10 in 'd.xlsx' and map the Divided_Code with corresponding names
d_df['Divided_Code'] = (d_df['Unnamed: 2'] // 10).astype(int)
combined_atlas_df['Code'] = combined_atlas_df['Class_2_Data'].str.extract(r'\((\d{2})')

# Step 2: Create a dictionary for mapping
code_to_name_mapping = dict(zip(combined_atlas_df['Code'], combined_atlas_df['Class_1_Data']))

# Step 3: Map the Divided_Code with the corresponding names
d_df['Replaced_Name'] = d_df['Divided_Code'].map(code_to_name_mapping)

# Save the modified DataFrame to a new Excel file
d_df.to_excel('updated_d.xlsx', index=False)

# Group by 'Year', 'Country', and 'Divided_Code', summing the 'Value' column for identical groups
grouped_e_df = d_df.groupby(['Year', 'Country', 'Divided_Code'], as_index=False).agg({
    'Value': 'sum',  # Summing the values for identical groups
    'Mapped_Name': 'first'  # Keeping the first Mapped_Name for each group
})

# Ensure all Divided_Code entries are two digits
grouped_e_df['Divided_Code'] = grouped_e_df['Divided_Code'].apply(lambda x: str(x).zfill(2))

# Map the Divided_Code in grouped_e_df to the corresponding name from combined_atlas_data
grouped_e_df['Mapped_Name'] = grouped_e_df['Divided_Code'].map(code_to_name_mapping)

# Save the updated DataFrame to a new Excel file
grouped_e_df.to_excel('mapped_grouped_e.xlsx', index=False)

# Optional: Display the result
print(grouped_e_df)


FileNotFoundError: [Errno 2] No such file or directory: 'ulkeler_with_codes.xlsx'

In [22]:
import pandas as pd

# Yıllar ve ilgili HS sürümleri
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı 
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Yeni ürünlerin bulunduğu Excel dosyasını yükle
new_products_df = pd.read_excel('2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')
new_products_df['Yıl'] = new_products_df['Yıl'].ffill()

# 2012 ve 2017'ye ait yeni ürünleri ayır
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]
new_products_2022_df = new_products_df[new_products_df['Yıl'] == 2022]

# Ülkeler dosyasını yükle
ulkeler_df = pd.read_excel('Ülkeler.xlsx')

# Sonuçları tutmak için bir DataFrame oluştur
sonuclar_df = pd.DataFrame()

# Her ülke ve her yıl için işlem yapma
for index, row in ulkeler_df.iterrows():
    ulke = row['Countries']  
    ulke_kodu = row['country_code']  
    
    for year in yillar:
        hs_version = hs_versiyonlari[year] 
        dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)
        
        # Veriyi yükle
        df = pd.read_csv(dosya_yolu)
        
        # Yalnızca ilgili ülkeye ait veriyi filtreleme
        ulke_veri = df[df['i'] == ulke_kodu]
        toplam_ticaret_hacmi = ulke_veri['v'].sum()
        
        # 2 basamaklı HS kodlarına göre ticaret hacmini hesaplama
        ulke_veri['2_Basamakli_HS_Kodu'] = ulke_veri['k'].astype(str).str[:2]  # 6 basamaklı HS kodunu 2 basamağa indirgeme
        hs_ticaret_hacmi = ulke_veri.groupby('2_Basamakli_HS_Kodu')['v'].sum().reset_index()

        # Veri türlerini uyumlu hale getirme (string dönüştürme)
        hs_ticaret_hacmi['2_Basamakli_HS_Kodu'] = hs_ticaret_hacmi['2_Basamakli_HS_Kodu'].astype(str)
        new_products_df['6 Basamaklı HS Kodu'] = new_products_df['6 Basamaklı HS Kodu'].astype(str)

        # HS kodlarının sektör adlarıyla eşleşmesi, new_products_df'den Sektör İsmi sütunu kullanılıyor
        hs_ticaret_hacmi = hs_ticaret_hacmi.merge(new_products_df[['6 Basamaklı HS Kodu', 'Sektör İsmi']], 
                                                  left_on='2_Basamakli_HS_Kodu', 
                                                  right_on='6 Basamaklı HS Kodu', 
                                                  how='left')

        # Sonuçları yıl, ülke ve sektör bazlı olarak ekleme
        for _, hs_row in hs_ticaret_hacmi.iterrows():
            sektor_adi = hs_row['Sektör İsmi']  # Sektör ismi new_products_df'ten alınır
            ticaret_hacmi = hs_row['v']

            # Yıl, ülke ve sektör bazlı sonuçları ekleme
            sonuclar_df.at[f'{ulke}_{year}', 'Yıl'] = year
            sonuclar_df.at[f'{ulke}_{year}', 'Ülke'] = ulke
            sonuclar_df.at[f'{ulke}_{year}', sektor_adi] = ticaret_hacmi

# Sonuçları Excel dosyasına kaydet
sonuclar_df.to_excel('CEE_MENA_Sektor_Bazinda_Yeni_Urun_Ticaret_Hacmi_2012_2021.xlsx', index=False)

# Sonuçları göster
print(sonuclar_df)


C:\Users\oguzo\AppData\Local\Temp\ipykernel_15508\1918690976.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ulke_veri['2_Basamakli_HS_Kodu'] = ulke_veri['k'].astype(str).str[:2]  # 6 basamaklı HS kodunu 2 basamağa indirgeme
C:\Users\oguzo\AppData\Local\Temp\ipykernel_15508\1918690976.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ulke_veri['2_Basamakli_HS_Kodu'] = ulke_veri['k'].astype(str).str[:2]  # 6 basamaklı HS kodunu 2 basamağa indirgeme
C:\Users\oguzo\AppData\Local\Temp\ipykernel_15508\19

                    Yıl        Ülke       NaN
Albania_2012     2012.0     Albania   449.430
Albania_2013     2013.0     Albania  4288.230
Albania_2014     2014.0     Albania   193.675
Albania_2015     2015.0     Albania   717.439
Albania_2016     2016.0     Albania   650.061
...                 ...         ...       ...
Bangladesh_2017  2017.0  Bangladesh   196.496
Bangladesh_2018  2018.0  Bangladesh   232.414
Bangladesh_2019  2019.0  Bangladesh   371.636
Bangladesh_2020  2020.0  Bangladesh   439.955
Bangladesh_2021  2021.0  Bangladesh   474.335

[500 rows x 3 columns]


C:\Users\oguzo\AppData\Local\Temp\ipykernel_15508\1918690976.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ulke_veri['2_Basamakli_HS_Kodu'] = ulke_veri['k'].astype(str).str[:2]  # 6 basamaklı HS kodunu 2 basamağa indirgeme


In [16]:
import pandas as pd

# Yıllar ve ilgili HS sürümleri
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı 
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Yeni ürünlerin bulunduğu Excel dosyasını yükle
new_products_df = pd.read_excel('2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')

# Sütun adlarını kontrol edelim
print(new_products_df.columns)

# 2012 ve 2017'ye ait yeni ürünleri ayır
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]

# Ülkeler dosyasını yükle
ulkeler_df = pd.read_excel('Ülkeler.xlsx')

# Sonuçları tutmak için bir DataFrame oluştur
sonuclar_df = pd.DataFrame()

# Her ülke ve her yıl için işlem yapma
for index, row in ulkeler_df.iterrows():
    ulke = row['Countries']  
    ulke_kodu = row['country_code']  
    
    for year in yillar:
        hs_version = hs_versiyonlari[year] 
        dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)
        
        # Veriyi yükle
        df = pd.read_csv(dosya_yolu)
        
        # Yalnızca ilgili ülkeye ait veriyi filtreleme
        ulke_veri = df[df['i'] == ulke_kodu]
        toplam_ticaret_hacmi = ulke_veri['v'].sum()
        
        # 2 basamaklı HS kodlarına göre ticaret hacmini hesaplama
        ulke_veri['2_Basamakli_HS_Kodu'] = ulke_veri['k'].astype(str).str[:2]  # 6 basamaklı HS kodunu 2 basamağa indirgeme
        hs_ticaret_hacmi = ulke_veri.groupby('2_Basamakli_HS_Kodu')['v'].sum().reset_index()

        # HS kodlarının sektör adlarıyla eşleşmesi
        hs_ticaret_hacmi = hs_ticaret_hacmi.merge(new_products_df, left_on='2_Basamakli_HS_Kodu', right_on='6 Basamaklı HS Kodu', how='left')

        # Sonuçları yıl, ülke ve sektör bazlı olarak ekleme
        for _, hs_row in hs_ticaret_hacmi.iterrows():
            sektor_adi = hs_row['General Sector Name']  # Eğer sektör adını öğrendiyseniz buraya koyun
            ticaret_hacmi = hs_row['v']

            # Yıl, ülke ve sektör bazlı sonuçları ekleme
            sonuclar_df.at[f'{ulke}_{year}', 'Yıl'] = year
            sonuclar_df.at[f'{ulke}_{year}', 'Ülke'] = ulke
            sonuclar_df.at[f'{ulke}_{year}', sektor_adi] = ticaret_hacmi

# Sonuçları Excel dosyasına kaydet
sonuclar_df.to_excel('CEE_MENA_Sektor_Bazinda_Yeni_Urun_Ticaret_Hacmi_2012_2021.xlsx', index=False)

# Sonuçları göster
print(sonuclar_df)


Index(['Yıl', '6 Basamaklı HS Kodu', 'Detaylı Sektör İsmi',
       '2 Basamaklı HS Kodu', 'Sektör İsmi'],
      dtype='object')


C:\Users\oguzo\AppData\Local\Temp\ipykernel_15508\3301223070.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ulke_veri['2_Basamakli_HS_Kodu'] = ulke_veri['k'].astype(str).str[:2]  # 6 basamaklı HS kodunu 2 basamağa indirgeme


ValueError: You are trying to merge on object and int64 columns for key '2_Basamakli_HS_Kodu'. If you wish to proceed you should use pd.concat